# Project 1 - Data Engineering
## 2 Data Exploration and Analysis
This notebook is for the implementation of task "2 Data Exploration and Analysis", as listed in the **Project Instructions**.

<div class="alert alert-success">
<b>Overview:</b><br>
Data Exploration and Analysis consists of the following parts worth 70 points:
<ul>
    <li><b>E1 - Obtain and Scrub</b> (15 points)</li>
    <li><b>E2 - Exploratory data analysis (EDA) </b>(20 points)</li>
    <li><b>E3 - Formulate hypotheses </b>(25 points)</li>
    <li><b>Follow the guidelines for Data Exploration and Analysis below </b>(10 points)</li>
</ul></div>

<div class="alert alert-success">
<b>Guidelines for Data Exploration and Analysis:</b><br>
    <ol>
        <li>Use a single Jupyer notebook for your project.</li>
        <li>Use only Python-code for your project.</li>
        <li>The use of automatic and semi-automatic data analyis tools is not allowed (e. g., PandasGUI, D-Tale, Mito, etc.). Only use packages we used in the coded lectures.</li>
        <li>Export your environment for submission as 'prj01-environment.txt'.</li>
        <li>Upload your resulting work as a zip file containing only a single jupyter notebook and required files to run the notebook. All cell outputs and <b>figures must display in jupyter lab</b>. (Test this, in particular when you use another environment like VS Code.)</li>
        <li>All code cells in your notebook must be runnable without errors or warnings (e. g., deprecated functions). Each error/warning subtracts -2 points (up to the full 10 points for following the  guidelines).</li>
        <li>Use only relative paths in your project.</li>
        <li>Avoid (excessive) code duplication.</li>
        <li>Avoid loops iterating over pandas objects (Series, DataFrames). Explicitly justify each exception via a comment. </li>
        <li>All coded steps in your analysis must be commented.</li>
        <li>Keep your code as well as outputs short, precise and readable. Each long or unnecessary output subtracts -2 points (up to the full 10 points for following the project guidelines).</li>
    </ol>
    <b>Late submissions are not accepted and earn you 0 points on the python project. </b>
</div>

Explicitly list which notebook toolset was used (jupyter lab/jupyter notebook/VS Code/etc.)

**here**: MY_TOOL, MY_BROWSER

Explicitly and clearly state the chosen dataset number and title:
### Bevölkerung ODÖ Hunde
#### Hundebestand seit 2002 - Bezirke Wien --> contains info about dog density per districts over years
#### Hunde pro Bezirk Wien --> contains info about dog breeds count per district in 2024
#### Hunderassen Wien --> contains info about dog breeds count per district 2012 - 2017


## E1 - Obtain and Scrub

### Obtain
Download the dataset and understand both:
- format: wide vs. long, separaters, decimal points, encoding, etc., and
- content: what variables are in the columns, what is their meaning?
To this end, identify and download metadata such as headers, category listings, explanatory reports, etc.
### Scrub
The aim of scrub is to create a clean version of the data for further analysis.
- Load the dataset and take care of dtypes (dates, numbers, categories, etc.). Justify why you don't load/use specific columns.
- Check for footnotes or any other notifications on special cell content, such as time series breaks. Follow up that information, and document your decision how to deal with it. Remember: A homework contained such info in the cell "76.1 b". The metadata defines what that "b" stands for.
- Choose an appropriate (Multi-)Index.
- Identify:
    1. missing values and get row and column overviews. Use graphical and/or numeric approaches. Once identified, handle missing values according to column type, time series property and data set size.
    2. duplicates (justify the used column subset). Remove duplicates - if any - and inspect what you removed.
- Transform to shape (tidy vs. wide) best suited for further analysis.
- Export the clean data to a file for inspection with an external data browser (e.g., MS Excel).
- Provide an overview of the clean dataset:
    1. show the dtypes
    2. quantitative column descriptions:
        1. categorical columns: number of unique values, counts
        2. numeric columns: range and median

In [1]:
# Importing necessary packages:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colormaps
import geopandas as gpd
import json
%matplotlib inline

## 1) Obtain the datasets and scrub data


#### 1) Dataset 1
Number of dogs (absolute and per 1,000 inhabitants) since 2002 - districts of Vienna
 	* NUTS | NUTS2-Region (Bundesland)
  * DISTRICT_CODE | Gemeindebezirkskennzahl (Schema: 9BBZZ, BB=Nummer Bezirk, ZZ=00)
  * SUB_DISTRICT_CODE | Zählbezirkskennzahl gemäß Stadt Wien (Schema: 9BBZZ, 9=Kennzeichnung Wien, BB=Nummer Bezirk, ZZ=Nummer Zählbezirk, **ZZ=99 bei fehlender Zählbezirkskennzeichnung**)
  * REF_YEAR | Referenzjahr
  * REF_DATE | Referenzdatum
  * DOG_VALUE | Anzahl der Hunde (absolut)
  * DOG_DENSITY | Anzahl der Hunde pro 1.000 EinwohnerInnen

Source: https://www.data.gv.at/katalog/dataset/1a48e78a-c660-41e0-ac32-8b675c0ea39c#additional-info

   TODO: add more description?

##### Obtain

In [2]:
# Import the first data set
dogs_2002 = pd.read_csv("vie-bez-biz-spo-dog-2002f.csv", sep=';', skiprows=1, dtype={"DOG_VALUE": "object"})
print("Dataframe shape: ", dogs_2002.shape)
dogs_2002.head(5)

Dataframe shape:  (528, 17)


,NUTS,DISTRICT_CODE,SUB_DISTRICT_CODE,REF_YEAR,REF_DATE,DOG_VALUE,DOG_DENSITY,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,AT13,90000,90000,2002,20020101,46.933,"29,87",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AT13,90100,90100,2002,20020101,542,"30,74",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AT13,90200,90200,2002,20020101,2.251,"25,29",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AT13,90300,90300,2002,20020101,1.904,"23,16",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AT13,90400,90400,2002,20020101,615,"21,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* The separator is ';'.
* The encoding is 'utf-8'.
* The original shape is (528, 17) - but many columns full of NaNs (later: cleaned up data => shape=(528, 7)).
* Long format:  
  * Contains entries for years by district of overall dog density sorted by years.
  * each row represents a single observation of district by year and dog breeds
* The first row contains the csv title. It is dropped and the second row is used for column titles.
* First column 'NUTS' (= identification and classification system for european states) contains always the same value, since 'AT13' stands for Vienna (and we only have data for Vienna in this dataset). We could drop it, but it also doesn't bother us too much right now
* DISTRICT_CODE '90000' stands for all districts combined
* Need to import DOG_VALUE as objects because the '.' in some values does not allow conversion to integer right now. Without specifying it, the values are recognised as floats.



##### Scrub

In [3]:
# check columns and their missing data ratio
((dogs_2002.isnull().sum() / len(dogs_2002)) * 100).sort_values(ascending=False)

Unnamed: 7           100.0
Unnamed: 15          100.0
Unnamed: 8           100.0
Unnamed: 14          100.0
Unnamed: 13          100.0
Unnamed: 12          100.0
Unnamed: 11          100.0
Unnamed: 10          100.0
Unnamed: 9           100.0
Unnamed: 16          100.0
NUTS                   0.0
DOG_VALUE              0.0
DOG_DENSITY            0.0
SUB_DISTRICT_CODE      0.0
DISTRICT_CODE          0.0
REF_YEAR               0.0
REF_DATE               0.0
dtype: float64

In [4]:
# drop all cols with NaN --> lots because of multiple 'unnecessary' seperators ';' in last columns
# other columns have no missing values
dogs_2002 = dogs_2002.dropna(axis=1, how='any')
print("Dataframe shape: ", dogs_2002.shape)

Dataframe shape:  (528, 7)


In [5]:
print(dogs_2002.dtypes)

NUTS                 object
DISTRICT_CODE         int64
SUB_DISTRICT_CODE     int64
REF_YEAR              int64
REF_DATE              int64
DOG_VALUE            object
DOG_DENSITY          object
dtype: object


Remove '.' in DOG_VALUE and convert to integer.

Change DOG_DENSITY to use '.' instead of ',' and the format to float to be able to calculate with it, if needed.

Change REF_DATE to format datetime since its currently int64.

In [6]:
dogs_2002['DOG_VALUE'] = dogs_2002['DOG_VALUE'].str.replace('.', '').astype(int)
dogs_2002['DOG_DENSITY'] = dogs_2002['DOG_DENSITY'].str.replace(',', '.').astype(float)
dogs_2002['REF_DATE'] = pd.to_datetime(dogs_2002['REF_DATE'].astype(str), format='%Y%m%d')

In [7]:
print(dogs_2002.dtypes)
dogs_2002.head(2)

NUTS                         object
DISTRICT_CODE                 int64
SUB_DISTRICT_CODE             int64
REF_YEAR                      int64
REF_DATE             datetime64[ns]
DOG_VALUE                     int64
DOG_DENSITY                 float64
dtype: object


,NUTS,DISTRICT_CODE,SUB_DISTRICT_CODE,REF_YEAR,REF_DATE,DOG_VALUE,DOG_DENSITY
0,AT13,90000,90000,2002,2002-01-01,46933,29.87
1,AT13,90100,90100,2002,2002-01-01,542,30.74


* District Code and Sub District Code are the same --> removing Sub District Code
* add Postal_CODE column for better comparison between datasets

In [8]:
# only execute if not already done (avoids errors)
if 'SUB_DISTRICT_CODE' in dogs_2002.columns:
    print(np.unique([dogs_2002['DISTRICT_CODE'] == dogs_2002['SUB_DISTRICT_CODE']], return_counts=True)) # check that columns are actually the same
    dogs_2002.drop(columns=['SUB_DISTRICT_CODE'], inplace=True)
dogs_2002["Postal_CODE"] = "1" + dogs_2002['DISTRICT_CODE'].astype(str).str[1:3] + "0"
dogs_2002.head(2) # Postal_CODE=1000 represents Vienna in total

(array([ True]), array([528]))


,NUTS,DISTRICT_CODE,REF_YEAR,REF_DATE,DOG_VALUE,DOG_DENSITY,Postal_CODE
0,AT13,90000,2002,2002-01-01,46933,29.87,1000
1,AT13,90100,2002,2002-01-01,542,30.74,1010


Check for duplicates --> looks good:

In [9]:
dogs_2002.duplicated(subset=['DISTRICT_CODE', 'REF_YEAR']).any()

np.False_

#### Dataset 2
  Number of registered dogs in Vienna per district and dog breed in the year 2024.

  * NUTS1 NUTS2 NUTS3: same as before, but NUTS3 in addition: provides district information for Vienna
  * DISTRICT_CODE: same structure as before
  * SUB_DISTRICT_CODE
  * Postal_CODE
  * Dog Breed
  * Anzahl
  * Ref_Date

Source: https://www.data.gv.at/katalog/dataset/71edef44-9d6c-4042-ab71-7207dc930ba7#additional-info

##### Obtain

In [10]:
# Import dataset 2
dog_types_2024 = pd.read_csv("hunde-wien.csv", sep=';', encoding='latin-1', skiprows=1)

print("Dataframe shape: ", dog_types_2024.shape)
dog_types_2024.head(5)

Dataframe shape:  (11117, 9)


,NUTS1,NUTS2,NUTS3,DISTRICT_CODE,SUB_DISTRICT_CODE,Postal_CODE,Dog Breed,Anzahl,Ref_Date
0,AT1,AT13,AT113,90100,.,1010,Afghanischer Windhund / Mischling,1.0,20240603
1,AT1,AT13,AT113,90100,.,1010,Akita / Belgischer Schäferhund,1.0,20240603
2,AT1,AT13,AT113,90100,.,1010,Alaskan Malamute,1.0,20240603
3,AT1,AT13,AT113,90100,.,1010,American Cocker Spaniel,2.0,20240603
4,AT1,AT13,AT113,90100,.,1010,American Cocker Spaniel / Kleinpudel Schwarz,1.0,20240603


* The separator is ';'
* The encoding is 'latin-1'.
* The shape is (11117, 9).
* Mixed breeds are also included, like 'Hovawart / Golden Retriever'.
* Format is long:
  * dog breeds can appear multiple times in breed-column. This is the case if at least two dogs of a breed are registered in different districts. If there is no dog of a specific breed registered in a distric, it is not listed in the dataset for this district.
  * Every district and breed combination has a single count value

##### Scrub

In [11]:
# check columns and their missing data ratio => no missing data
((dog_types_2024.isnull().sum() / len(dog_types_2024)) * 100).sort_values(ascending=False)

NUTS1                0.0
NUTS2                0.0
NUTS3                0.0
DISTRICT_CODE        0.0
SUB_DISTRICT_CODE    0.0
Postal_CODE          0.0
Dog Breed            0.0
Anzahl               0.0
Ref_Date             0.0
dtype: float64

In [12]:
dog_types_2024.dtypes.to_frame().T

,NUTS1,NUTS2,NUTS3,DISTRICT_CODE,SUB_DISTRICT_CODE,Postal_CODE,Dog Breed,Anzahl,Ref_Date
0,object,object,object,int64,object,int64,object,float64,int64


Looks like some columns only have 1 unique value => NUTS1-3, SUB_DISTRICT_CODE and Ref_Date to be exact. Also, no district is missing, which is good.

In [13]:
dog_types_2024.nunique()

NUTS1                   1
NUTS2                   1
NUTS3                   1
DISTRICT_CODE          23
SUB_DISTRICT_CODE       1
Postal_CODE            23
Dog Breed            3481
Anzahl                152
Ref_Date                1
dtype: int64

In [14]:
print("DISTRICT_CODE:", dog_types_2024["DISTRICT_CODE"].unique())
print("Postal_CODE:", dog_types_2024["Postal_CODE"].unique())

DISTRICT_CODE: [90100 90200 90300 90400 90500 90600 90700 90800 90900 91000 91100 91200
 91300 91400 91500 91600 91700 91800 91900 92000 92100 92200 92300]
Postal_CODE: [1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140
 1150 1160 1170 1180 1190 1200 1210 1220 1230]


Drop NUTS1, NUTS2, NUTS3 and SUB_DISTRICT CODE. Don't want to drop Ref_Date yet, maybe could be used for some insights with the above data frame.

In [15]:
dog_types_2024 = dog_types_2024.drop(['NUTS1', 'NUTS2', 'NUTS3', 'SUB_DISTRICT_CODE'], axis=1)
dog_types_2024['Ref_Date'] = pd.to_datetime(dog_types_2024['Ref_Date'].astype(str), format='%Y%m%d')
dog_types_2024.dtypes.to_frame().T

,DISTRICT_CODE,Postal_CODE,Dog Breed,Anzahl,Ref_Date
0,int64,int64,object,float64,datetime64[ns]


The 'Anzahl'-column contains one value that currently cannot be converted to an integer without losing information. It seems that there are 1012 dogs of breed 'Unbekannt / Mischling', which was recognised as a float with decimal places:

In [16]:
dog_types_2024[(dog_types_2024["Anzahl"] != dog_types_2024["Anzahl"].astype(int))]

,DISTRICT_CODE,Postal_CODE,Dog Breed,Anzahl,Ref_Date
10314,92200,1220,Unbekannt / Mischling,1.012,2024-06-03


In [17]:
dog_types_2024.loc[10314, "Anzahl"] = int(str(dog_types_2024.loc[10314, "Anzahl"]).replace('.','')) # could apply this type conversion on whole column to be future proof for future data
(dog_types_2024["Anzahl"] == dog_types_2024["Anzahl"].astype(int)).unique() # now ready to be formatted to int

array([ True])

In [18]:
dog_types_2024["Anzahl"] = dog_types_2024["Anzahl"].astype("int")
print(dog_types_2024.dtypes) # we could also change dtype for 'Dog Breed' to 'category', but 3500 categories are a bit much. We leave it for now
dog_types_2024.head(2)

DISTRICT_CODE             int64
Postal_CODE               int64
Dog Breed                object
Anzahl                    int64
Ref_Date         datetime64[ns]
dtype: object


,DISTRICT_CODE,Postal_CODE,Dog Breed,Anzahl,Ref_Date
0,90100,1010,Afghanischer Windhund / Mischling,1,2024-06-03
1,90100,1010,Akita / Belgischer Schäferhund,1,2024-06-03


Check if some Dog Breeds have duplicates per district --> looks good.

In [19]:
dog_types_2024.duplicated(subset=['Dog Breed', 'Postal_CODE']).any()

np.False_

Check if there are unknown Dog Breeds:
  * There appear to be in total 50 entries in the dataset with at least partly unknown Dog Breeds.
  * There are in total 8200 dogs with an at least partly uknown dog breed.
  * The dataset has only 6 unique values in the 'Dog Breed'-column that contain an unknown dog breed.
  * They are kept for the moment.

In [20]:
def check_for_unknown_dog_breeds(df: pd.DataFrame):
    bool_unknown = df['Dog Breed'].str.contains('Unbekannt', case=False)
    print("Total number of entries with unknown dog breed:", bool_unknown.sum())
    print("Total number of dogs with at least partly unknown dog breed:", df[bool_unknown]["Anzahl"].sum())
    tmp_array = df[bool_unknown]['Dog Breed'].unique()
    print("Number of unknown dog breeds:", len(tmp_array))
    print("Unknown dog breeds:", tmp_array)

check_for_unknown_dog_breeds(dog_types_2024)

Total number of entries with unknown dog breed: 50
Total number of dogs with at least partly unknown dog breed: 8200
Number of unknown dog breeds: 6
Unknown dog breeds: ['Unbekannt' 'Unbekannt / Mischling' 'Unbekannt / Dackel'
 'Unbekannt / Kleiner Münsterländer' 'Unbekannt / Mudi'
 'Unbekannt / Pit Bull Terrier']


In [21]:
dog_types_2024.head()

,DISTRICT_CODE,Postal_CODE,Dog Breed,Anzahl,Ref_Date
0,90100,1010,Afghanischer Windhund / Mischling,1,2024-06-03
1,90100,1010,Akita / Belgischer Schäferhund,1,2024-06-03
2,90100,1010,Alaskan Malamute,1,2024-06-03
3,90100,1010,American Cocker Spaniel,2,2024-06-03
4,90100,1010,American Cocker Spaniel / Kleinpudel Schwarz,1,2024-06-03


We could also drop the postal_code column, since it basically provides the same data as the district_code column:

In [22]:
(dog_types_2024["DISTRICT_CODE"].astype(str).str[1:3] == dog_types_2024["Postal_CODE"].astype(str).str[1:3]).unique()

array([ True])

#### Dataset 3
Number of registered dogs in Vienna per district and dog breed in the years 2012 to 2017. 

The layout of the dataset is basically the same as the one before, we only have data for multiple years.

  * NUTS1: AT1
  * NUTS2: AT13
  * NUTS3: AT113
  * DISTRICT_CODE: Bezirke, Format 9BB00
  * SUB_DISTRICT_CODE: Zählbezirk, leer
  * Postal_CODE: Postleitzahl, Format 1BB0
  * Dog Breed: Hunderasse
  * Anzahl: Anzahl der jeweiligen Hunderasse
  * Ref_Date: Jahr

Source: https://www.data.gv.at/katalog/dataset/31f526f5-2cf2-49f8-a63d-0c63a0f87a3e

In [23]:
# Import dataset 3
dog_types_2012 = pd.read_csv("hunde-vie.csv", sep=';', encoding='latin-1', skiprows=1)

print("Dataframe shape: ", dog_types_2012.shape)
dog_types_2012.head(5)

Dataframe shape:  (33793, 9)


,NUTS1,NUTS2,NUTS3,DISTRICT_CODE,SUB_DISTRICT_CODE,Postal_CODE,Dog Breed,Anzahl,Ref_Date
0,AT1,AT13,AT113,90100,.,1010,Afghanischer Windhund,1,20123112
1,AT1,AT13,AT113,90100,.,1010,Amerikanischer Cockerspaniel,1,20123112
2,AT1,AT13,AT113,90100,.,1010,Amerikanischer Staffordshire-Terrier,2,20123112
3,AT1,AT13,AT113,90100,.,1010,Australian Shepherd Dog,2,20123112
4,AT1,AT13,AT113,90100,.,1010,Australian Terrier,1,20123112


* The separator is ';'
* The encoding is 'latin-1'.
* The shape is (33793, 9).
* Format is long:
  * dog breeds can appear multiple times in breed-column. This is the case if at least two dogs of a breed are registered in different districts or in different years. If there is no dog of a specific breed registered in a distric, it is not listed in the dataset for this district.
  * Every district, ref_date and breed combination has a single count value


Repeat steps as with dataframe 2, since they have the same format.

In [24]:
# check columns and their missing data ratio => no missing data
((dog_types_2012.isnull().sum() / len(dog_types_2012)) * 100).sort_values(ascending=False)

NUTS1                0.0
NUTS2                0.0
NUTS3                0.0
DISTRICT_CODE        0.0
SUB_DISTRICT_CODE    0.0
Postal_CODE          0.0
Dog Breed            0.0
Anzahl               0.0
Ref_Date             0.0
dtype: float64

Looks like some columns only have 1 unique value => NUTS1-3 and SUB_DISTRICT_CODE to be exact. Also, no district is missing, which is good. We have 6 different years in the dataset:

In [25]:
dog_types_2012.nunique()

NUTS1                   1
NUTS2                   1
NUTS3                   1
DISTRICT_CODE          23
SUB_DISTRICT_CODE       1
Postal_CODE            23
Dog Breed            1286
Anzahl                337
Ref_Date                6
dtype: int64

Column Ref_Date contains more entries than above, lets see which years are included:

In [26]:
dog_types_2012['Ref_Date'].value_counts()

Ref_Date
20173112    5886
20163112    5822
20153112    5768
20143112    5691
20133112    5501
20123112    5125
Name: count, dtype: int64

In [27]:
print(dog_types_2012.groupby("Ref_Date")["DISTRICT_CODE"].nunique()) # no districts missing in each year
print(dog_types_2012.groupby("Ref_Date")["Postal_CODE"].nunique())

Ref_Date
20123112    23
20133112    23
20143112    23
20153112    23
20163112    23
20173112    23
Name: DISTRICT_CODE, dtype: int64
Ref_Date
20123112    23
20133112    23
20143112    23
20153112    23
20163112    23
20173112    23
Name: Postal_CODE, dtype: int64


In [28]:
dog_types_2012.dtypes.to_frame().T

,NUTS1,NUTS2,NUTS3,DISTRICT_CODE,SUB_DISTRICT_CODE,Postal_CODE,Dog Breed,Anzahl,Ref_Date
0,object,object,object,int64,object,int64,object,int64,int64


'Anzahl' is already an int this time => no correction necessary.

Change Ref_Date from int64 to date (important: format is now different to before: yyyy**dd**mm, not yyyymm**dd**):

In [29]:
dog_types_2012 = dog_types_2012.drop(['NUTS1', 'NUTS2', 'NUTS3', 'SUB_DISTRICT_CODE'], axis=1)
dog_types_2012['Ref_Date'] = pd.to_datetime(dog_types_2012['Ref_Date'].astype(str), format='%Y%d%m') # convert int to real datetime
print(dog_types_2012.dtypes.to_frame().T)

  DISTRICT_CODE Postal_CODE Dog Breed Anzahl        Ref_Date
0         int64       int64    object  int64  datetime64[ns]


Check for duplicates and unkown dog breeds --> has only the unknown dog breed 'Unbekannt' and no duplicates too.
* Unkown can stay for now, even though their number is very high with 98.707 dogs (even if we consider that the dataset contains multiple years of data).



In [30]:
dog_types_2012.duplicated(subset=['Dog Breed', 'Postal_CODE', 'Ref_Date']).any()

np.False_

In [31]:
check_for_unknown_dog_breeds(dog_types_2012)

Total number of entries with unknown dog breed: 138
Total number of dogs with at least partly unknown dog breed: 98707
Number of unknown dog breeds: 1
Unknown dog breeds: ['Unbekannt']


### Merging Dataset 2 and 3 as they contain the same info for different years

In [32]:
print(dog_types_2024.dtypes.to_frame().T)
print(dog_types_2012.dtypes.to_frame().T)

  DISTRICT_CODE Postal_CODE Dog Breed Anzahl        Ref_Date
0         int64       int64    object  int64  datetime64[ns]
  DISTRICT_CODE Postal_CODE Dog Breed Anzahl        Ref_Date
0         int64       int64    object  int64  datetime64[ns]


In [33]:
combined_dog_types = pd.concat([dog_types_2012, dog_types_2024]) # dataframes have same columns => concat
print("Shape:", combined_dog_types.shape)

Shape: (44910, 5)


In [34]:
combined_dog_types_by_year_and_district = combined_dog_types.pivot_table(index=['Ref_Date', 'Postal_CODE'], columns='Dog Breed', values='Anzahl', fill_value=0).sort_values(by='Ref_Date', ascending=False)
combined_dog_types_by_year_and_district.head(2)

Dog Breed               Affenpinscher  Affenpinscher / Afghanischer Windhund  \
Ref_Date   Postal_CODE                                                         
2024-06-03 1230                   0.0                                    0.0   
           1120                   1.0                                    0.0   

Dog Breed               Affenpinscher / Beagle  \
Ref_Date   Postal_CODE                           
2024-06-03 1230                            0.0   
           1120                            0.0   

Dog Breed               Affenpinscher / Border Terrier  \
Ref_Date   Postal_CODE                                   
2024-06-03 1230                                    0.0   
           1120                                    0.0   

Dog Breed               Affenpinscher / Cairn Terrier  \
Ref_Date   Postal_CODE                                  
2024-06-03 1230                                   0.0   
           1120                                   0.0   

Dog Breed               Affenpinscher / Griffon belge  \
Ref_Date   Postal_CODE                                  
2024-06-03 1230                                   0.0   
           1120                                   0.0   

Dog Breed               Affenpinscher / Mischling  \
Ref_Date   Postal_CODE                              
2024-06-03 1230                               2.0   
           1120                               0.0   

Dog Breed               Affenpinscher / Scottish Terrier  \
Ref_Date   Postal_CODE                                     
2024-06-03 1230                                      0.0   
           1120                                      0.0   

Dog Breed               Affenpinscher / Shih Tzu  \
Ref_Date   Postal_CODE                             
2024-06-03 1230                              0.0   
           1120                              0.0   

Dog Breed               Affenpinscher / Zwergschnauzer pfeffer-salz  ...  \
Ref_Date   Postal_CODE                                               ...   
2024-06-03 1230                                                 0.0  ...   
           1120                                                 0.0  ...   

Dog Breed               Österreichischer Pinscher / Prager Rattler  \
Ref_Date   Postal_CODE                                               
2024-06-03 1230                                                0.0   
           1120                                                0.0   

Dog Breed               Österreichischer Pinscher / Rauhhaar Dachshund Normal  \
Ref_Date   Postal_CODE                                                          
2024-06-03 1230                                                       0.0       
           1120                                                       0.0       

Dog Breed               Österreichischer Pinscher / Shar Pei  \
Ref_Date   Postal_CODE                                         
2024-06-03 1230                                          0.0   
           1120                                          0.0   

Dog Breed               Österreichischer Pinscher / Spitz  \
Ref_Date   Postal_CODE                                      
2024-06-03 1230                                       0.0   
           1120                                       0.0   

Dog Breed               Österreichischer Pinscher / Tibetan Spaniel  \
Ref_Date   Postal_CODE                                                
2024-06-03 1230                                                 0.0   
           1120                                                 0.0   

Dog Breed               Österreichischer Pinscher / Weimaraner Kurzhaar  \
Ref_Date   Postal_CODE                                                    
2024-06-03 1230                                                     0.0   
           1120                                                     0.0   

Dog Breed               Österreichischer Pinscher / Whippet  \
Ref_Date   Postal_CODE                                        

Sanity Check: 

Values are the same (apart from 0) --> looks good. 

In [35]:
unique_values = []
unique_values_single = []
for col in combined_dog_types_by_year_and_district.columns:
    unique_values.extend(combined_dog_types_by_year_and_district[col].unique())

unique_values_single.extend(dog_types_2012['Anzahl'].unique())
unique_values_single.extend(dog_types_2024['Anzahl'].unique())
unique_values_single = set(unique_values_single)
unique_values = set(unique_values)
differences = unique_values.symmetric_difference(unique_values_single)
print(differences)

{np.float64(0.0)}


## E2 - Exploratory data analysis (EDA)
Use the clean dataset and understand and explore the relationships in the data (numerical, visual, statistical). This includes at least but is not limited to:
- A comprehensive textual description of meaning for relevant fields in the dataset
- Statistical/numerical descriptions and visualization techniques we learned in the course inluding correlations, distributions and groupings of variing degrees.
- Checks for data quality, e. g., completeness, plausibility, outliers
- Handling any identified problems with the data
- If necessary, use additional data wrangling in line with your EDA and only keep what's necessary for the following steps of your analysis with appropriate data granularity and form ("tidy data")

Don't:
- test/prove hypotheses here. EDA should only motivate hypotheses.

Advanced/bonus:
- Depending on your hypothesis you may want to join external data (e.g., merge external highest education level to existing vaccination data) for additional insights.

Explain all steps taken and your thinking why you deem them necessary.

In [36]:
# No missing data - TODO: heatmaps still necessary?


BOXPLOTS

BARPLOTS

In [ ]:
# Plot: Absolute number of dogs of the top 30 breeds (across all districts in 2024)
sum_dog_types_2024 = pd.DataFrame(dog_types_2024.groupby("Dog Breed")["Anzahl"].sum())
top30_sum_dog_types_2024 = sum_dog_types_2024.sort_values(by="Anzahl", ascending=False).head(30).reset_index()
bool_unbekannt_top30_2024 = top30_sum_dog_types_2024["Dog Breed"].str.contains("Unbekannt", case=False)

fig, ax = plt.subplots(figsize=(10,7))
sns.barplot(top30_sum_dog_types_2024, x="Anzahl", y="Dog Breed", color="blue", ax=ax)
sns.despine(ax=ax, trim=True)
for i in np.arange(len(top30_sum_dog_types_2024))[bool_unbekannt_top30_2024]:
    ax.patches[i].set_color('orange')
ax.set_title("Absolute number of dogs of the top 30 breeds (across all districts in 2024)\n[Unknown marked in orange]", loc="left")
ax.grid(axis="x")

In [127]:
df_top10_2024 = dog_types_2024.groupby(["Postal_CODE", "Dog Breed"]).agg(count = ("Anzahl", "sum")).sort_values(["count"], ascending=False).groupby(["Postal_CODE"]).head(10) # double groupby and agg using sum-function to avoid deprecation warning of other methods
df_top10_2024 = df_top10_2024.reset_index()

In [ ]:
# plot: total number of dogs per district and year
# TODO: how to handle jump between 2018 and 2024?
df_years_district = pd.DataFrame(combined_dog_types.groupby(["Ref_Date", "Postal_CODE"])["Anzahl"].count().reset_index())
px.line(df_years_district, x="Ref_Date", y="Anzahl", color='Postal_CODE', markers=True)

In [ ]:
# plot: total number of dogs per district and year
df_years_district["BEZNR"] = df_years_district["Postal_CODE"].astype(str).str[1:3].astype("int32").astype(str)
df_years_district["year"] = df_years_district["Ref_Date"].dt.strftime("%Y").astype("int32")
with open('vienna_.geojson', 'r') as file:
    geo_json = json.load(file)
fig = px.choropleth(df_years_district,
                    geojson=geo_json,
                    color="Anzahl",
                    animation_frame='year',
                    locations="BEZNR",
                    featureidkey="properties.cartodb_id",
                    projection="mercator",
                    color_continuous_scale="deep",
                    title="Total number of dogs per district and year"
                   ) # TODO: maybe use log scaling
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=500,width=500)
fig.show()

In [ ]:
# plot: absolute number of dogs since 2002
geo_dogs_2002 = dogs_2002.copy()
geo_dogs_2002["BEZNR"] = geo_dogs_2002["DISTRICT_CODE"].astype(str).str[1:3].astype("int32").astype(str)
fig = px.choropleth(geo_dogs_2002,
                    geojson=geo_json,
                    color=np.log2(geo_dogs_2002["DOG_VALUE"]), # TODO: maybe use log scaling
                    animation_frame='REF_YEAR',
                    locations="BEZNR",
                    featureidkey="properties.cartodb_id",
                    projection="mercator",
                    color_continuous_scale="deep",
                    title="absolute number of dogs since 2002"
                   ) 
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=500,width=500)
"""
fig.update_layout(coloraxis_colorbar=dict(
    title="Absolute number",
    tickvals=[6,7,8,9],
    ticktext=["", "10M", "100M", "1B"],
))
"""
fig.show()
# TODO: maybe concentrate on some specific districts

## E3 - Formulate hypotheses

*Note: Read this section entirely and understand it - every group member.*

A hypothesis is an idea or explanation for something that is based on known facts but has not yet been proved. A hypothesis is a compact, concise statement, such as: "Individuals with higher income have (on average) more offspring.", that will be answered based on facts (the data). https://gradcoach.com/what-is-a-research-hypothesis-or-scientific-hypothesis/

Formulate *N* non-trivial hypotheses, 1 per group member, and regard the following criteria:
- State the hypothesis explicitly in concise language.
- The hypothesis must be **motivated** by either **EDA results** or **literature** (citation in the report needed).
- The hypothesis must refer to **endpoints** that are **testable**. Specifically, the endpoint must be derived from the data.
- Think of real-life use cases/consequences of your results (textual description).
- For each hypothesis explain all executed steps.
- In case of extreme or implausible results check the validity of your data.
- For each hypothesis export the artifacts (figures, tables, etc.) required for the report.
- If you decide to use a statistical test, use it properly. In particular, check the validity and comparability of the samples.

Do not:
- State nebulous, vague hypotheses. These don't contain endpoints and are unclear to test (i.e., answer).
- Use post-hoc hypotheses. Portraying an empirically inspired **post hoc hypothesis as a priori** violates the **falsification principle** crucial for hypothesis-driven (that is, confirmatory) empirical research. Falsification is severe scientific fraud.
- State trivial hypotheses (e.g., hypthesis 2: "Not Hypothesis 1").
- Answer based on "common knowledge".
- Try to **produce positively tested hypotheses**. If a well motivated hypothesis is negative, this is an important finding (see Simpson's Paradox). The value of a tested hypothesis lies in the information or learning it provides.

Example: The homework with Simpson's Paradox. The pooled overall comparison between the genders would be the EDA motivating the hypothesis: "At UC Berkeley the by-department admissions rate for females is lower than for males." It should be tested using samples of department admission rates for the 2 **groups** male and female. No steps of the test should be done in EDA (or a priory to stating the hypothesis). The groups should be compared graphically, e.g., via a stripplot overlayed with a boxplot. The figure should be labelled properly and exported for the report. A (paired) t-test **may** be used (it's optional) to test this hypothesis statistically. For different data (e.g., time series) different approaches may be required. You don't have to use statistical tests, in particular if you don't know what they are doing.


### E3-H1: "The prevalence of specific dog breeds in Vienna is strongly influenced by real estate prices."
Rationale: Higher real estate prices may attract certain dog breeds, indicating a potential link between the types of dogs found in high-income areas and local housing costs.
Data Needed: Dog breed data ('hunde-vie.csv'/hunde-wien.csv), Real estate prices dataset.

Author: Carlos Eduardo Tichy


### E3-H2: "Dog ownership trends are significantly different between high-cost and low-cost real estate areas of Vienna."
Rationale: Real estate prices could affect the type of dog breeds owned, changes in real estate prices could correlate with changes in dog breeds owned. Data Needed: Dog breed data ('hunde-vie.csv'/hunde-wien.csv), Real estate prices dataset.


Author: Theresa Spiel



### E3-H3: "xxx"
Author: Group member 3


### E3-H4: "xxx"
Author: Group member 4


### E3-H5: "xxx"
Author: Group member 5
